In [21]:
import cv2
import dlib
import numpy as np

import matplotlib.pyplot as plt
import math
import os
from PIL import Image

import time
import torch
import tensorflow as tf
from tracker import Tracker
from keras import backend as K
from timeit import default_timer as timer
from model import Generator
from torch.autograd import Variable
from torchvision.transforms import ToTensor, ToPILImage

In [22]:
# Dlib load detect face
# cnn_face_detector = dlib.cnn_face_detection_model_v1("../weights/mmod_human_face_detector.dat")
# hog_face_detector = dlib.get_frontal_face_detector()
face_cascade = cv2.CascadeClassifier('../weights/haarcascade_frontalface_default.xml')

In [23]:
# path = "/mnt/24eb92b8-4fc1-4f32-b05c-deaa999ce6cf/Documents/datasets/face/camera_supervisor/1_02_cut.avi"
path = "/home/damvantai/Videos/Webcam/2018-08-31-165523.webm"
cap = cv2.VideoCapture(path)
font = cv2.FONT_HERSHEY_SIMPLEX

In [24]:
tracker = Tracker(iou_thresh=0.3, max_frames_to_skip=20,
                 max_trace_length=15, trackIdCount=0)
track_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
                    (0, 255, 255), (255, 0, 255), (255, 127, 255),
                    (127, 0, 255), (127, 0, 127)] 

In [25]:
# obj_type = []
# for i

In [26]:
for i in range(3):
    print(i)

0
1
2


In [27]:
def convert_x_to_bbox(x, score=None):
    w = np.sqrt(x[2] * x[3])
    h = x[2] / w
    if w == 0.0:
        print('Stop: ', x)
    return np.array([x[0] - w / 2., x[1] - h / 2., x[0] + w / 2., x[1] + h / 2.], dtype=np.float32).reshape((1, 4))


In [28]:
a = [[1, 2, 3, 4]]

In [29]:
convert_x_to_bbox(a[0])

array([[-0.73205084,  1.5669873 ,  2.732051  ,  2.4330127 ]],
      dtype=float32)

In [30]:
num_frame = 0
start = time.time()
while cap.isOpened():
    obj_type = []
    num_frame += 1
    ret, frame = cap.read()
    
#     faces_cnn = cnn_face_detector(frame, 1)
#     faces_hog = hog_face_detector(frame, 1)
#     for face in faces_hog:
#         x1 = face.left()
#         y1 = face.top()
#         x2 = face.right()
#         y2 = face.bottom()
        
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Use cascade (best is default cascade)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 3)
        
    for i in range(len(faces)):
        obj_type.append("face")
        faces[i] = convert_x_to_bbox(faces[i])
    
    print("type(faces): ", type(faces))
    print(faces)
    
    if len(faces) > 0:
#         print(faces[0][0], faces[0][1], faces[0][2], faces[0][3])
        print("Number tracker.tracks: ", len(tracker.tracks))
        print("tracks: ", tracker.tracks)
        tracker.Update(faces, obj_type)
#         print(len(tracker.tracks))
        for i in range(len(tracker.tracks)):
            print("Number tracker.tracks: ", len(tracker.tracks))
            if len(tracker.tracks[i].trace) > 1:
                print(len(tracker.tracks[i].trace))
                for j in range(len(tracker.tracks[i].trace) - 1):
                    x1 = tracker.tracks[i].trace[j][0][0]
                    y1 = tracker.tracks[i].trace[j][1][0]
                    x2 = tracker.tracks[i].trace[j + 1][0][0]
                    y2 = tracker.tracks[i].trace[j + 1][1][0]
                    print("x1, y1: ", x1, y1)
                    print("x2, y2: ", x2, y2)
                    clr = tracker.tracks[i].track_id % 9
                    cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)),
                             track_colors[clr], 2)
                    cv2.circle(frame, (int(x1), int(y1)), 3, track_colors[clr], 2)
        
    end = time.time()
    FPS = num_frame / float(end - start)    
    cv2.putText(frame, str(FPS), (0, 30), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow("frame", frame)
    
    k = cv2.waitKey(1) & 0xff
    if k == ord('n'):
        continue
    elif k == 27:
        break
cap.release()
cv2.destroyAllWindows()      

type(faces):  <class 'numpy.ndarray'>
[[251 445 333 446]]
Number tracker.tracks:  0
tracks:  []
Number tracker.tracks:  1
type(faces):  <class 'numpy.ndarray'>
[[252 446 330 447]]
Number tracker.tracks:  1
tracks:  [<tracker.Track object at 0x7fa458037ac8>]
Number tracker.tracks:  1
type(faces):  <class 'tuple'>
()
type(faces):  <class 'numpy.ndarray'>
[[251 445 333 446]]
Number tracker.tracks:  1
tracks:  [<tracker.Track object at 0x7fa458037ac8>]
Number tracker.tracks:  1
2
x1, y1:  291.1428527832031 446.3571472167969
x2, y2:  291.75 445.75
type(faces):  <class 'numpy.ndarray'>
[[249 445 331 446]]
Number tracker.tracks:  1
tracks:  [<tracker.Track object at 0x7fa458037ac8>]
Number tracker.tracks:  1
3
x1, y1:  291.1428527832031 446.3571472167969
x2, y2:  291.75 445.75
x1, y1:  291.75 445.75
x2, y2:  290.39544677734375 445.5564880371094
type(faces):  <class 'numpy.ndarray'>
[[251 445 333 446]]
Number tracker.tracks:  1
tracks:  [<tracker.Track object at 0x7fa458037ac8>]
Number tracker

type(faces):  <class 'numpy.ndarray'>
[[362 310 576 311]]
Number tracker.tracks:  6
tracks:  [<tracker.Track object at 0x7fa458037ac8>, <tracker.Track object at 0x7fa458037a58>, <tracker.Track object at 0x7fa4580aa6a0>, <tracker.Track object at 0x7fa458037be0>, <tracker.Track object at 0x7fa458037f98>, <tracker.Track object at 0x7fa4580432e8>]
Number tracker.tracks:  6
12
x1, y1:  291.1428527832031 446.3571472167969
x2, y2:  291.75 445.75
x1, y1:  291.75 445.75
x2, y2:  290.39544677734375 445.5564880371094
x1, y1:  290.39544677734375 445.5564880371094
x2, y2:  291.45166015625 445.49700927734375
x1, y1:  291.45166015625 445.49700927734375
x2, y2:  291.3863220214844 445.4313659667969
x1, y1:  291.3863220214844 445.4313659667969
x2, y2:  291.32098388671875 445.36572265625
x1, y1:  291.32098388671875 445.36572265625
x2, y2:  291.2556457519531 445.3000793457031
x1, y1:  291.2556457519531 445.3000793457031
x2, y2:  291.1903076171875 445.23443603515625
x1, y1:  291.1903076171875 445.234436035

x1, y1:  290.9942932128906 445.0375061035156
x2, y2:  290.928955078125 444.97186279296875
x1, y1:  290.928955078125 444.97186279296875
x2, y2:  290.8636169433594 444.9062194824219
x1, y1:  290.8636169433594 444.9062194824219
x2, y2:  290.79827880859375 444.840576171875
x1, y1:  290.79827880859375 444.840576171875
x2, y2:  290.7329406738281 444.7749328613281
x1, y1:  290.7329406738281 444.7749328613281
x2, y2:  290.6676025390625 444.70928955078125
Number tracker.tracks:  7
11
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
x1, y1:  294.5 448.5
x2, y2:  294.5 448.5
Number tracker.tracks:  7
10
x1, y1:  493.5 311.5
x2, y2:  493.5 311.5
x1, y1:  493.5 311.5
x2, y2: 

type(faces):  <class 'numpy.ndarray'>
[[339 315 565 316]]
Number tracker.tracks:  8
tracks:  [<tracker.Track object at 0x7fa458037ac8>, <tracker.Track object at 0x7fa458037a58>, <tracker.Track object at 0x7fa4580aa6a0>, <tracker.Track object at 0x7fa458037be0>, <tracker.Track object at 0x7fa458037f98>, <tracker.Track object at 0x7fa4580432e8>, <tracker.Track object at 0x7fa458043f98>, <tracker.Track object at 0x7fa4580378d0>]
Number tracker.tracks:  9
16
x1, y1:  291.3863220214844 445.4313659667969
x2, y2:  291.32098388671875 445.36572265625
x1, y1:  291.32098388671875 445.36572265625
x2, y2:  291.2556457519531 445.3000793457031
x1, y1:  291.2556457519531 445.3000793457031
x2, y2:  291.1903076171875 445.23443603515625
x1, y1:  291.1903076171875 445.23443603515625
x2, y2:  291.1249694824219 445.1687927246094
x1, y1:  291.1249694824219 445.1687927246094
x2, y2:  291.05963134765625 445.1031494140625
x1, y1:  291.05963134765625 445.1031494140625
x2, y2:  290.9942932128906 445.0375061035156

type(faces):  <class 'numpy.ndarray'>
[[336 317 552 318]]
Number tracker.tracks:  9
tracks:  [<tracker.Track object at 0x7fa458037ac8>, <tracker.Track object at 0x7fa458037a58>, <tracker.Track object at 0x7fa4580aa6a0>, <tracker.Track object at 0x7fa458037be0>, <tracker.Track object at 0x7fa458037f98>, <tracker.Track object at 0x7fa4580432e8>, <tracker.Track object at 0x7fa458043f98>, <tracker.Track object at 0x7fa4580378d0>, <tracker.Track object at 0x7fa458037b70>]
Number tracker.tracks:  9
16
x1, y1:  291.2556457519531 445.3000793457031
x2, y2:  291.1903076171875 445.23443603515625
x1, y1:  291.1903076171875 445.23443603515625
x2, y2:  291.1249694824219 445.1687927246094
x1, y1:  291.1249694824219 445.1687927246094
x2, y2:  291.05963134765625 445.1031494140625
x1, y1:  291.05963134765625 445.1031494140625
x2, y2:  290.9942932128906 445.0375061035156
x1, y1:  290.9942932128906 445.0375061035156
x2, y2:  290.928955078125 444.97186279296875
x1, y1:  290.928955078125 444.97186279296875


...

x1, y1:  443.5 314.5
x2, y2:  443.5 314.5
x1, y1:  443.5 314.5
x2, y2:  443.5 314.5
x1, y1:  443.5 314.5
x2, y2:  443.5 314.5
x1, y1:  443.5 314.5
x2, y2:  443.5 314.5
x1, y1:  443.5 314.5
x2, y2:  443.5 314.5
Number tracker.tracks:  11
13
x1, y1:  439.78570556640625 317.3571472167969
x2, y2:  438.6428527832031 317.9285888671875
x1, y1:  438.6428527832031 317.9285888671875
x2, y2:  437.5 318.5000305175781
x1, y1:  437.5 318.5000305175781
x2, y2:  436.3571472167969 319.07147216796875
x1, y1:  436.3571472167969 319.07147216796875
x2, y2:  433.2847595214844 319.54132080078125
x1, y1:  433.2847595214844 319.54132080078125
x2, y2:  431.7294616699219 320.0910339355469
x1, y1:  431.7294616699219 320.0910339355469
x2, y2:  430.1741638183594 320.6407470703125
x1, y1:  430.1741638183594 320.6407470703125
x2, y2:  428.6188659667969 321.1904602050781
x1, y1:  428.6188659667969 321.1904602050781
x2, y2:  427.0635986328125 321.74017333984375
x1, y1:  427.0635986328125 321.74017333984375
x2, y2:  425

type(faces):  <class 'numpy.ndarray'>
[[356 320 566 321]]
Number tracker.tracks:  12
tracks:  [<tracker.Track object at 0x7fa458037f98>, <tracker.Track object at 0x7fa458043f98>, <tracker.Track object at 0x7fa458037b70>, <tracker.Track object at 0x7fa4580359b0>, <tracker.Track object at 0x7fa458037be0>, <tracker.Track object at 0x7fa458043198>, <tracker.Track object at 0x7fa458037cf8>, <tracker.Track object at 0x7fa458053048>, <tracker.Track object at 0x7fa458037c88>, <tracker.Track object at 0x7fa458037dd8>, <tracker.Track object at 0x7fa4580432b0>, <tracker.Track object at 0x7fa4580357b8>]
Number tracker.tracks:  12
16
x1, y1:  417.52496337890625 309.6234130859375
x2, y2:  415.78900146484375 309.7131042480469
x1, y1:  415.78900146484375 309.7131042480469
x2, y2:  414.05303955078125 309.80279541015625
x1, y1:  414.05303955078125 309.80279541015625
x2, y2:  425.97802734375 309.64520263671875
x1, y1:  425.97802734375 309.64520263671875
x2, y2:  424.665283203125 309.72723388671875
x1, y1

type(faces):  <class 'numpy.ndarray'>
[[396 328 591 329]]
Number tracker.tracks:  12
tracks:  [<tracker.Track object at 0x7fa458037f98>, <tracker.Track object at 0x7fa458037b70>, <tracker.Track object at 0x7fa4580359b0>, <tracker.Track object at 0x7fa458037be0>, <tracker.Track object at 0x7fa458043198>, <tracker.Track object at 0x7fa458037cf8>, <tracker.Track object at 0x7fa458053048>, <tracker.Track object at 0x7fa458037c88>, <tracker.Track object at 0x7fa458037dd8>, <tracker.Track object at 0x7fa4580432b0>, <tracker.Track object at 0x7fa4580357b8>, <tracker.Track object at 0x7fa458043be0>]
Number tracker.tracks:  12
16
x1, y1:  414.05303955078125 309.80279541015625
x2, y2:  425.97802734375 309.64520263671875
x1, y1:  425.97802734375 309.64520263671875
x2, y2:  424.665283203125 309.72723388671875
x1, y1:  424.665283203125 309.72723388671875
x2, y2:  423.35247802734375 309.80926513671875
x1, y1:  423.35247802734375 309.80926513671875
x2, y2:  422.0397033691406 309.89129638671875
x1, y1

type(faces):  <class 'numpy.ndarray'>
[[402 320 611 321]]
Number tracker.tracks:  11
tracks:  [<tracker.Track object at 0x7fa458037f98>, <tracker.Track object at 0x7fa458037b70>, <tracker.Track object at 0x7fa458043198>, <tracker.Track object at 0x7fa458037cf8>, <tracker.Track object at 0x7fa458053048>, <tracker.Track object at 0x7fa458037c88>, <tracker.Track object at 0x7fa458037dd8>, <tracker.Track object at 0x7fa4580432b0>, <tracker.Track object at 0x7fa4580357b8>, <tracker.Track object at 0x7fa458043be0>, <tracker.Track object at 0x7fa458037ba8>]


KeyboardInterrupt: 